# Student Score analytics

## Data Load

In [4]:
import pandas as pd
import zipfile

In [2]:
DATA_ADDR = 'https://www.kaggle.com/datasets/bhavikjikadara/student-study-performance/download?datasetVersionNumber=1'

In [5]:
myzip = zipfile.ZipFile(DATA_ADDR)

OSError: [Errno 22] Invalid argument: 'https://www.kaggle.com/datasets/bhavikjikadara/student-study-performance/download?datasetVersionNumber=1'

In [3]:
pd.read_excel(DATA_ADDR)

ValueError: Excel file format cannot be determined, you must specify an engine manually.